In [10]:
import pandas as pd
import glob

In [11]:
files = [item for item in glob.glob(r'/Users/yaroslav/Desktop/Excel_file/*{}'.format('.xlsx'))]
files

['/Users/yaroslav/Desktop/Excel_file/statistics-16.xlsx',
 '/Users/yaroslav/Desktop/Excel_file/statistics-15.xlsx']

In [169]:
combined = pd.DataFrame()
for file in files:
    file = pd.read_excel(file, 
                         skiprows = 5, usecols=['ОМСУ','Номер Добродела','№ ЕДС','Дата обращения', 
                                                'Адрес заявки','Срок исполнения', 'Текущий статус заявки',
                                                'Текущий статус заявки',
                                                'Текст заявки', 'Ответ УК предоставлен?'])
    file = file [['ОМСУ','Номер Добродела','№ ЕДС','Дата обращения', 
                                                'Адрес заявки','Срок исполнения', 'Текущий статус заявки',
                                                  'Текущий статус заявки', 
                                                'Текст заявки', 'Ответ УК предоставлен?']]
    
    file.columns = ['ОМСУ','Номер Добродела','№ ЕДС','Дата обращения', 
                                                'Адрес заявки','Срок исполнения', 'Текущий статус заявки',
                                                  'k', 
                                                'Текст заявки', 'Ответ УК предоставлен?']
    combined = pd.concat([combined, file])
    combined

/Users/yaroslav/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/yaroslav/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [170]:
combined

,ОМСУ,Номер Добродела,№ ЕДС,Дата обращения,Адрес заявки,Срок исполнения,Текущий статус заявки,k,Текст заявки,Ответ УК предоставлен?
0,Солнечногорск г о,7627550.0,26803-14-09-2022-574,14.09.2022 08:31,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",22.09.2022 08:31,В работе,В работе,"Номер Добродела: 7627550\nд. Голубое, Тверецки...",-
1,Солнечногорск г о,7621526.0,26803-12-09-2022-573,12.09.2022 20:39,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",13.09.2022 20:39,Ожидает подтверждения,Ожидает подтверждения,Номер Добродела: 7621526\nУК не может устранит...,Уважаемый Сергей Владимирович! Следы масла на ...
2,Солнечногорск г о,7610721.0,26803-09-09-2022-572,09.09.2022 20:28,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",12.09.2022 20:28,Ожидает подтверждения,Ожидает подтверждения,Номер Добродела: 7610721\nНет горячей воды. Из...,Уважаемая Светлана Сергеевна! Горячее водоснаб...
3,Солнечногорск г о,7610541.0,26803-09-09-2022-571,09.09.2022 19:07,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",15.09.2022 14:12,Контроль ГЖИ: Просрочено,Контроль ГЖИ: Просрочено,Номер Добродела: 7610541\nСломался маленький л...,Уважаемая Светлана Сергеевна! Лифт выведен из ...
4,Солнечногорск г о,7584267.0,26803-05-09-2022-570,05.09.2022 12:32,"Солнечногорск г, Голубое д, Тверецкий пр-д, 20",06.09.2022 12:32,Закрыто,Закрыто,"Номер Добродела: 7584267\nпервый подъезд, 20 д...",Уважаемая Наталья Сергеевна! Заявка выполнена.
5,Солнечногорск г о,7588841.0,26803-05-09-2022-569,05.09.2022 12:21,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",20.09.2022 12:21,Требуется доработка,Требуется доработка,Номер Добродела: 7588841\nНеобходимо произвест...,Уважаемая Светлана Сергеевна! Ремонт напольной...
6,Солнечногорск г о,7587666.0,26803-05-09-2022-568,05.09.2022 12:15,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",20.09.2022 12:15,В работе,В работе,Номер Добродела: 7587666\nОтваливается фасадна...,-
7,Солнечногорск г о,7568164.0,26803-30-08-2022-567,30.08.2022 16:18,"Солнечногорск г, Голубое д, Тверецкий пр-д, 20",07.09.2022 16:18,Закрыто,Закрыто,Номер Добродела: 7568164\nтребуется ремонт в п...,Уважаемая Наталья Сергеевна! Заявка выполнена.
8,Солнечногорск г о,7564632.0,26803-30-08-2022-566,30.08.2022 09:54,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",04.09.2022 09:54,Закрыто,Закрыто,Номер Добродела: 7564632\nЗдравствуйте. 18.08....,Уважаемая Татьяна Васильевна! Сварочные работы...
9,Солнечногорск г о,7564637.0,26803-30-08-2022-565,30.08.2022 09:54,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",31.08.2022 09:54,Закрыто,Закрыто,Номер Добродела: 7564637\nЗдравствуйте. 18.08....,Уважаемый Татьяна Васильевна! Сварочные работы...


In [171]:
status_in_table=[]
status_need = ['В работе', 'Контроль ГЖИ: Просрочено', 'Новая заявка', 'Срок превышен', 
               'Требуется доработка', 'Требуется доработка (c просрочкой)','Контроль ГЖИ: Устранение']
for i in combined['Текущий статус заявки']:
    if i not in status_need and i not in status_in_table: 
        status_in_table.append(i)
print(status_in_table)

['Ожидает подтверждения', 'Закрыто', 'Закрыто (после контроля ГЖИ)', 'Контроль ГЖИ: Получен ответ УК', 'Закрыто. Отправлено в ДД']


In [154]:
# combined = combined[combined['Текущий статус заявки'].
#                     str.contains("Ожидает подтверждения|Закрыто|Закрыто после контроля ГЖИ")==False]

In [172]:
combined = combined.set_index('k')
combined


,ОМСУ,Номер Добродела,№ ЕДС,Дата обращения,Адрес заявки,Срок исполнения,Текущий статус заявки,Текст заявки,Ответ УК предоставлен?
k,,,,,,,,,
В работе,Солнечногорск г о,7627550.0,26803-14-09-2022-574,14.09.2022 08:31,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",22.09.2022 08:31,В работе,"Номер Добродела: 7627550\nд. Голубое, Тверецки...",-
Ожидает подтверждения,Солнечногорск г о,7621526.0,26803-12-09-2022-573,12.09.2022 20:39,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",13.09.2022 20:39,Ожидает подтверждения,Номер Добродела: 7621526\nУК не может устранит...,Уважаемый Сергей Владимирович! Следы масла на ...
Ожидает подтверждения,Солнечногорск г о,7610721.0,26803-09-09-2022-572,09.09.2022 20:28,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",12.09.2022 20:28,Ожидает подтверждения,Номер Добродела: 7610721\nНет горячей воды. Из...,Уважаемая Светлана Сергеевна! Горячее водоснаб...
Контроль ГЖИ: Просрочено,Солнечногорск г о,7610541.0,26803-09-09-2022-571,09.09.2022 19:07,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",15.09.2022 14:12,Контроль ГЖИ: Просрочено,Номер Добродела: 7610541\nСломался маленький л...,Уважаемая Светлана Сергеевна! Лифт выведен из ...
Закрыто,Солнечногорск г о,7584267.0,26803-05-09-2022-570,05.09.2022 12:32,"Солнечногорск г, Голубое д, Тверецкий пр-д, 20",06.09.2022 12:32,Закрыто,"Номер Добродела: 7584267\nпервый подъезд, 20 д...",Уважаемая Наталья Сергеевна! Заявка выполнена.
Требуется доработка,Солнечногорск г о,7588841.0,26803-05-09-2022-569,05.09.2022 12:21,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",20.09.2022 12:21,Требуется доработка,Номер Добродела: 7588841\nНеобходимо произвест...,Уважаемая Светлана Сергеевна! Ремонт напольной...
В работе,Солнечногорск г о,7587666.0,26803-05-09-2022-568,05.09.2022 12:15,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",20.09.2022 12:15,В работе,Номер Добродела: 7587666\nОтваливается фасадна...,-
Закрыто,Солнечногорск г о,7568164.0,26803-30-08-2022-567,30.08.2022 16:18,"Солнечногорск г, Голубое д, Тверецкий пр-д, 20",07.09.2022 16:18,Закрыто,Номер Добродела: 7568164\nтребуется ремонт в п...,Уважаемая Наталья Сергеевна! Заявка выполнена.
Закрыто,Солнечногорск г о,7564632.0,26803-30-08-2022-566,30.08.2022 09:54,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",04.09.2022 09:54,Закрыто,Номер Добродела: 7564632\nЗдравствуйте. 18.08....,Уважаемая Татьяна Васильевна! Сварочные работы...


In [173]:
combined.drop(status_in_table, axis=0,inplace = True)
combined

,ОМСУ,Номер Добродела,№ ЕДС,Дата обращения,Адрес заявки,Срок исполнения,Текущий статус заявки,Текст заявки,Ответ УК предоставлен?
k,,,,,,,,,
В работе,Солнечногорск г о,7627550.0,26803-14-09-2022-574,14.09.2022 08:31,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",22.09.2022 08:31,В работе,"Номер Добродела: 7627550\nд. Голубое, Тверецки...",-
Контроль ГЖИ: Просрочено,Солнечногорск г о,7610541.0,26803-09-09-2022-571,09.09.2022 19:07,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",15.09.2022 14:12,Контроль ГЖИ: Просрочено,Номер Добродела: 7610541\nСломался маленький л...,Уважаемая Светлана Сергеевна! Лифт выведен из ...
Требуется доработка,Солнечногорск г о,7588841.0,26803-05-09-2022-569,05.09.2022 12:21,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",20.09.2022 12:21,Требуется доработка,Номер Добродела: 7588841\nНеобходимо произвест...,Уважаемая Светлана Сергеевна! Ремонт напольной...
В работе,Солнечногорск г о,7587666.0,26803-05-09-2022-568,05.09.2022 12:15,"Солнечногорск г, Голубое д, Тверецкий пр-д, 17",20.09.2022 12:15,В работе,Номер Добродела: 7587666\nОтваливается фасадна...,-
В работе,Краснознаменск г о,7626405.0,12293-13-09-2022-765,13.09.2022 19:15,"Краснознаменск г, Строителей ул, 12/2",18.09.2022 19:15,В работе,Номер Добродела: 7626405\nДобрый день. В перво...,-


In [113]:
combined.to_excel(r'/Users/yaroslav/Desktop/Excel_file/res1.xlsx')